# Cvičení 4a

Tématem tohoto cvičení jsou přímé metody řešení soustav lineárních rovnic. Budeme se zabývat LDLT rozkladem.

## LDLT rozklad

LDLT rozklad je vhodný pro symetrické matice. Lze jej popsat následujícím pseudokódem:

```
n = size(A)
L = I, D = 0

v(1) = A(1, 1)
D(1, 1) = v(1)
L(2:n, 1) = A(2:n, 1)/v(1)

for j = 2 to n do
    for i = 1 to j - 1 do
        v(i) = L(j, i)D(i, i)
    end

    v(j) = A(j, j) - L(j, 1:j-1)v(1:j-1)
    D(j, j) = v(j)
    L(j+1:n, j) = (A(j+1:n, j) - L(j+1:n, 1:j-1)v(1:j-1)) / v(j)
end
```

V následující části implementujete LDLT rozklad a využijete ho k řešení soustavy se symetrickou maticí.

In [ ]:
# ÚKOL: Doplňte následující kód

import numpy as np

def my_ldlt(A):
    """
    Vypočítá LDLT rozklad symetrické matice.
    Vstup: A - A symetrická matice
    Výstup: L, D - Matice dekompozice takové, že platí A = L*D*L.T
    """
    m, n = A.shape

    # Zkontrolujme, že je matice čtvercová
    if m != n:
        raise ValueError('Matice musí být čtvercová!')

    L = np.eye(m)
    D = np.zeros((m, m))
    v = np.zeros(m)

    # doplňte
    v[0] = A[0, 0]
    D[0, 0] = v[0]
    L[1:n, 0] = A[1:n, 0] / v[0]

    # opraveno pomocí ChatGPT
    for j in range(1, n):
        for i in range(j):
            v[i] = L[j, i] * D[i, i]
        
        v[j] = A[j, j] - np.sum(L[j, 0:j]**2 * np.diag(D)[0:j])
        D[j, j] = v[j]
        L[j+1:n, j] = (A[j+1:n, j] - L[j+1:n, 0:j] @ (L[j, 0:j] * np.diag(D)[0:j])) / v[j]

    return L, D

A = np.array([[1, 2, 3],
              [2, 3, 4],
              [3, 4, 5]])
L, D = my_ldlt(A)
print(L)
print(D)
print(L @ D @ L.T)

[[1. 0. 0.]
 [2. 1. 0.]
 [3. 2. 1.]]
[[ 1.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0.  0.]]
[[1. 2. 3.]
 [2. 3. 4.]
 [3. 4. 5.]]


In [85]:
# Vygenerujeme náhodnou symetrickou matici
A = np.random.rand(5, 5)

A = A + A.transpose()
print(A)

[[1.87629338 1.13938557 1.3588787  0.43637334 1.54752448]
 [1.13938557 1.35006779 0.44769368 1.0492905  1.13195737]
 [1.3588787  0.44769368 0.45926768 0.70400899 1.11769176]
 [0.43637334 1.0492905  0.70400899 0.4914827  0.81236356]
 [1.54752448 1.13195737 1.11769176 0.81236356 1.41729995]]


In [68]:
# Otestujeme, že váš kód vrací správný výsledek
L, D = my_ldlt(A)
AA = L@D@L.transpose()
print(AA-A)

[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 0.00000000e+00  4.44089210e-16  0.00000000e+00  0.00000000e+00
  -1.66533454e-16]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  6.66133815e-16
   1.11022302e-16]
 [ 0.00000000e+00  2.22044605e-16  8.88178420e-16 -6.66133815e-16
   2.77555756e-16]
 [ 0.00000000e+00 -2.22044605e-16  0.00000000e+00 -9.99200722e-16
  -2.22044605e-16]]


Známe-li rozklad symetrické matice $\mathsf{A} = \mathsf{L}\mathsf{D}\mathsf{L}^T$, můžeme jej využít k řešení soustavy $\mathsf{A}\mathbf{x} = \mathbf{b}$, kterou převedeme na 
$$\mathsf{L}\mathsf{D}\mathsf{L}^T\mathbf{x} = \mathbf{b}$$
a vyřešíme vhodnými substitucemi s využitím dopředné a zpětné substituce.

Rozmyslete si, jak takové substituce budou vypadat a v následující části vyřešte náhodnou soustavu.

In [69]:
# ÚKOL: Zkopírujte z minulého cvičení

def fsubst(L, b):
    m, n = L.shape
    x = np.zeros(n)

    x[0] = b[0] / L[0, 0]

    for i in range(1, m):
        suma = 0
        for j in range(i):
            suma += L[i, j] * x[j]

        x[i] = (b[i] - suma)/L[i, i]

    return x

In [70]:
# ÚKOL: Zkopírujte z minulého cvičení

def bsubst(U, b):
    m, n = U.shape
    x = np.zeros(n)

    x[-1] = b[-1] / U[-1, -1]

    for i in range(m-1, -1, -1):
        suma = 0
        for j in range(m-1, i, -1):
            suma += U[i, j] * x[j]
        
        x[i] = (b[i] - suma)/U[i, i]
    
    return x

In [86]:
import scipy
import scipy.linalg

# ÚKOL: Doplňte následující buňku a vyřešte soustavu rovnic Ax = b se symetrickou náhodnou maticí A a
# náhodným vektorem pravé strany b. Využijte vaše metody fsubst a bsubst
b = np.random.rand(5)

# doplňte řešení soustavy Ax = b pomoci LDLT rozkladu
L, D, P = scipy.linalg.ldl(A)
AA = L @ D @ L.T
print(L)
print(AA - A)

y = fsubst(L, b)
y_s_vlnkou = np.zeros(b.size)
for i in range(b.size):
    y_s_vlnkou[i] = y[i] / D[i, i]
x = bsubst(L.T, y_s_vlnkou)

print(x)
print(A @ x)
print(b)

[[ 1.          0.          0.          0.          0.        ]
 [ 0.60725342  1.          0.          0.          0.        ]
 [ 0.72423573 -0.57354318  1.          0.          0.        ]
 [ 0.23257202  1.19163576 -1.13004641  1.          0.        ]
 [ 0.82477745  0.29204802 -0.1445458   0.85664795  1.        ]]
[[0.00000000e+00 0.00000000e+00 2.22044605e-16 0.00000000e+00
  2.22044605e-16]
 [0.00000000e+00 0.00000000e+00 5.55111512e-17 0.00000000e+00
  2.22044605e-16]
 [2.22044605e-16 1.66533454e-16 2.22044605e-16 1.11022302e-16
  2.22044605e-16]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [2.22044605e-16 0.00000000e+00 2.22044605e-16 0.00000000e+00
  2.22044605e-16]]
[ 0.28843816 -0.72833698  1.50041137  2.26044496 -1.74915867]
[0.02974709 0.40895305 0.3913244  0.10794957 0.65613674]
[0.02974709 0.40895305 0.3913244  0.10794957 0.65613674]


In [87]:
# Porovnáme vaše řešení a řešení pomocí numpy
print(x)
print(np.linalg.solve(A, b))

[ 0.28843816 -0.72833698  1.50041137  2.26044496 -1.74915867]
[ 0.28843816 -0.72833698  1.50041137  2.26044496 -1.74915867]
